# USEFUL FOR PULLING ENVIRONMENTAL DATA ACROSS WHOLE REGIONS

In [1]:
import os
import geopandas as gpd
import pandas as pd
import ee
import geemap
import gee_data_acq as gda

In [2]:
# Authenticate to Earth Engine
ee.Authenticate()

True

In [2]:
# Authenticate to Earth Engine
#ee.Authenticate()
ee.Initialize()

In [3]:
folder_directory = 'D:/!!Research/!!!Data/ArcGIS_Projects'
raster_working = f'{folder_directory}/quick_new_poly/files_for_python'

### Importing CSV of bands to export

In [4]:
bands_csv_name = 'images_and_bands.csv'
images_and_bands = pd.read_csv(f'{raster_working}/{bands_csv_name}')
images_and_bands.head()

,collection,collection_loc,bands,band_rename,base_re,unit,resample,resample_res,resample_method
0,clay_0_5,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_0_5,30,m,True,50,bilinear
1,clay_5_15,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_5_15,30,m,True,50,bilinear
2,clay_15_30,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_15_30,30,m,True,50,bilinear
3,clay_30_60,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_30_60,30,m,True,50,bilinear
4,clay_60_100,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_60_100,30,m,True,50,bilinear


### Prepping other bands to download

In [5]:
#shrubs_states = ['california', 'kansas', 'new_mexico', 'nevada', 'texas']
#sts_abbvs = ['CA', 'KS', 'NM', 'NV', 'TX']
sts_abbrv = ['TX', 'NM']

In [6]:
other_bands = []

elevation = ee.Image('USGS/3DEP/10m').select(['elevation'])
other_bands.append(elevation)
slope = ee.Terrain.slope(elevation).select(['slope']).rename(['slope'])
other_bands.append(slope)
aspect = ee.Terrain.aspect(elevation).select(['aspect']).rename(['aspect'])
other_bands.append(aspect)

nhd_area_tx = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[0]}/NHDArea")
nhd_flowline_tx = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[0]}/NHDFlowline")
nhd_tx = nhd_area_tx.merge(nhd_flowline_tx)
nhd_area_nm = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[1]}/NHDArea")
nhd_flowline_nm = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[1]}/NHDFlowline")
nhd_nm = nhd_area_nm.merge(nhd_flowline_nm)
nhd = nhd_tx.merge(nhd_nm)
dist_nhd = nhd.distance(searchRadius=10000, maxError=50) #originally searchRadius = 50000
#dist_nhd = dist_nhd.clip(ee_items[num]) #change clip area
dist_nhd = dist_nhd.select(['distance'])
dist_nhd = dist_nhd.rename('dist_drain')
other_bands.append(dist_nhd)

#roads = ee.FeatureCollection('TIGER/2016/Roads')
#dist_roads = roads.distance(searchRadius=50000, maxError=50)
#dist_roads = dist_roads.select(['distance'])
#dist_roads = dist_roads.rename('dist_road')
#other_bands.append(dist_roads)

print(other_bands)

[<ee.image.Image object at 0x0000022F63698B30>, <ee.image.Image object at 0x0000022F5DBB96A0>, <ee.image.Image object at 0x0000022F6369F8C0>, <ee.image.Image object at 0x0000022F637784A0>]


In [7]:
image_stack_builder = gda.ImageStackBuilder(images_and_bands)
rast_crs = 'EPSG:6350'
image_stack = image_stack_builder.build_image_stack(other_bands, rast_crs = rast_crs)
print(image_stack.bandNames().getInfo())

['clay_0_5', 'clay_5_15', 'clay_15_30', 'clay_30_60', 'clay_60_100', 'clay_100_200', 'sand_0_5', 'sand_5_15', 'sand_15_30', 'sand_30_60', 'sand_60_100', 'sand_100_200', 'silt_0_5', 'silt_5_15', 'silt_15_30', 'silt_30_60', 'silt_60_100', 'silt_100_200', 'bd_0_5', 'bd_5_15', 'bd_15_30', 'bd_30_60', 'bd_60_100', 'bd_100_200', 'om_0_5', 'om_5_15', 'om_15_30', 'om_30_60', 'om_60_100', 'om_100_200', 'ph_0_5', 'ph_5_15', 'ph_15_30', 'ph_30_60', 'ph_60_100', 'ph_100_200', 'ksat_0_5', 'ksat_5_15', 'ksat_15_30', 'ksat_30_60', 'ksat_60_100', 'ksat_100_200', 'theta_r_0_5', 'theta_r_5_15', 'theta_r_15_30', 'theta_r_30_60', 'theta_r_60_100', 'theta_r_100_200', 'theta_s_0_5', 'theta_s_5_15', 'theta_s_15_30', 'theta_s_30_60', 'theta_s_60_100', 'theta_s_100_200', 'caco3', 'cec', 'ec', 'sar', 'kw_025', 'resdept', 'soil_depth', 'wind_erodibility', 'rf_025', 'water_storage', 'ann_rain', 'temp_range', 'elevation', 'slope', 'aspect', 'dist_drain']


### Testing exporting imagery

In [8]:
# no slash needed at end of directory
shp_dir = f'{raster_working}/fishnet_cells'

In [11]:
#shp_dir = f'{folder_directory}shp_file'
# Don't need to repeate the folder directory in an fstring, just need to tack on the
# additional directories that lead to the folder you want to store the data in.
# e.g. 'test_mesq_prediction_points/acc_points_env_data/env_data_no_ls'
data_folder = 'env_data'

# needs the overarching directory, the folder to put the data into, the stack of images to download, and
# the folder containing the polygon shapefiles
shp_data = gda.ImageryDownload(raster_working, data_folder, image_stack, shp_dir)
shp_data.process_shp_files(scale = 50)

Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\quick_new_poly\files_for_python\env_data\cell_0_env_image.tif
Export task started for fishnet_cell_0.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\quick_new_poly\files_for_python\env_data\cell_1_env_image.tif
Export task started for fishnet_cell_1.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\quick_new_poly\files_for_python\env_data\cell_2_env_image.tif
Export task started for fishnet_cell_10.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\quick_new_poly\files_for_python\env_data\cell_3_env_image.tif
Export task started for fishnet_cell_11.shp.
Generating URL ...
Please wait ...
Data downloaded to D:\!!Research\!!!Data\ArcGIS_Projects\quick_new_poly\files_for_python\env_data\cell_4_env_image.tif
Export task started for fishnet_cell_12.shp.
Generat